<a href="https://colab.research.google.com/github/vasantvohra/Wiki-Summary/blob/master/getting_relations_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install spacy

In [0]:
!pip install plac

In [0]:
!pip install datefinder

In [0]:
from spacy import displacy

In [0]:
TEXTS=['On Thursday there was a massive U.S. aerial bombardment in which more than 300 Tomahawk cruise missiles rained down on Baghdad.',
      'Earlier Saturday ,Baghdad was again targeted ..',]

In [0]:
m=["Eight of the attacks occurred in South Mumbai at Chhatrapati Shivaji Terminus",
    "The Oberoi Trident,The Taj Palace & Tower, Leopold Cafe, Cama Hospital" ,
    "The Nariman House Jewish community centre, the Metro Cinema",
    "and in a lane behind the Times of India building and St. Xavier's College.There was also an explosion at Mazagaon",
    "in Mumbai's port area, and in a taxi at Vile Parle. By the early morning of 28 November, all sites except for the Taj Hotel had been secured by the Mumbai Police Department and security forces. On 29 November, India's National Security Guards (NSG) conducted 'Operation Black Tornado' to flush out the remaining attackers; it culminated in the death of the last remaining attackers at the Taj Hotel and ended the attacks"]

In [0]:
nlp = spacy.load('en')

In [0]:
#doc = nlp(TEXTS)
#print([token.text for token in doc[2].lefts])

In [0]:
# coding: utf8
"""A simple example of extracting relations between phrases and entities using
spaCy's named entity recognizer and the dependency parse. Here, we extract
money and currency values (entities labelled as MONEY) and then check the
dependency tree to find the noun phrase they are referring to – for example:
$9.4 million --> Net income.
Compatible with: spaCy v2.0.0+
"""
from __future__ import unicode_literals, print_function

import plac
import spacy


TEXTs = [
    'Net income was $9.4 million compared to the prior year of $2.7 million.',
    'Revenue exceeded twelve billion dollars, with a loss of $1b.',]



# model=("Model to load (needs parser and NER)", "positional", None, str))
def main(model='en_core_web_sm'):
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
    print("Processing %d texts" % len(TEXTS))

    for text in TEXTs:
        doc = nlp(text)
        relations = extract_currency_relations(doc)
        for r1, r2 in relations:
            print('{:<10}\t{}\t{}'.format(r2.text, r2.ent_type_, r1.text))


def extract_currency_relations(doc):
    # merge entities and noun chunks into one token
    spans = list(doc.ents) + list(doc.noun_chunks) #main nouns
    for span in spans:
        span.merge()

    relations = []
    for money in filter(lambda w: w.ent_type_ == 'MONEY', doc): #taking entity money only
        print('1',money)
        if money.dep_ in ('attr', 'dobj'): #money attr=link between verb and it's complement , dobj= (gave raise)
            subject = [w for w in money.head.lefts if w.dep_ == 'nsubj'] 
            print('2',subject)
            if subject:
                subject = subject[0]
                relations.append((subject, money))
        elif money.dep_ == 'pobj' and money.head.dep_ == 'prep': #head gives connector word
            relations.append((money.head.head, money))
    return relations
print(main())

Loaded model 'en_core_web_sm'
Processing 2 texts
1 $9.4 million
2 [Net income]
1 $2.7 million
$9.4 million	MONEY	Net income
$2.7 million	MONEY	the prior year
1 twelve billion dollars
2 [Revenue]
1 1b
twelve billion dollars	MONEY	Revenue
1b        	MONEY	a loss
None


In [0]:
displacy.render(nlp(str(TEXTS)), style='dep', jupyter = True, options = {'distance': 120})


In [0]:
text = ["On Tuesday, one women ran on the street while 2 men ran and 1 child ran on the sidewalk. Also, there were 3 people walking.", 
        "One person was walking yesterday, but there were 2 women running as well as 4 men and 3 kids running.", 
        "The other day, there were three women running and also 6 men and 5 kids running on the sidewalk. Also, there were 2 people walking in the park."]

nlp = spacy.load('en')
docs = [nlp(t) for t in text]
for i, doc in enumerate(docs):
    for j, sent in enumerate(doc.sents):
        subjects = [w for w in sent if w.dep_ == 'nsubj']
        for subject in subjects:
            numbers = [w for w in subject.lefts if w.dep_ == 'nummod']
            if len(numbers) == 1:
                print('document.sentence: {}.{}, subject: {}, action: {}, numbers: {}'.format(i, j, subject.text, subject.head.text, numbers[0].text))

document.sentence: 0.0, subject: women, action: ran, numbers: one
document.sentence: 0.0, subject: men, action: ran, numbers: 2
document.sentence: 0.0, subject: child, action: ran, numbers: 1
document.sentence: 1.0, subject: person, action: walking, numbers: One


In [0]:
docs = [nlp(t) for t in TEXTS]
r=[]
for i, doc in enumerate(docs):
    for j, sent in enumerate(doc.sents):
        subjects = [w for w in sent if w.dep_ == 'attr']
        print(subjects)
        for subject in subjects:
          gpe=[w for w in subject.lefts if w.pos_=='PROPN']
          print(gpe)
          ge=[w for w in subject.lefts if w.dep_=='amod' and w.pos_=='ADJ']
          he=[w for w in subject.head.lefts if w.dep_=='prep']
          print(he)
          for h in he:
            da=[w for w in h.rights]
            print(da)
          if gpe:
            gpe=gpe[0]
            #ge=[w for w in gpe.rights if w.pos_=='ADJ']
          if ge:
            a=ge[1]
          print(he[0],da[0],gpe,a,subjects[0])
        #  print(subject.head)
       
  

In [0]:
docs = [nlp(t) for t in TEXTS]
r=[]
for i, doc in enumerate(docs):
    for j, sent in enumerate(doc.sents):
        subjects = [w for w in sent if w.dep_ == 'nsubjpass' ]
        print(subjects)
        for subject in subjects:
          gpe=[w for w in sent if w.dep_ == 'npadvmod' ]
          print(gpe)
          for g in gpe:
            ge=[w for w in g.lefts if w.dep_=='amod']
            #he=[w for w in subject.head.lefts if w.dep_=='prep']
            #print(he)
            #for h in he:
             # da=[w for w in h.rights]
              #print(da)
          #if gpe:
            #gpe=gpe[0]
            #ge=[w for w in gpe.rights if w.pos_=='ADJ']
          #if ge:
           # a=ge[1]
          print(ge[0],gpe[0],subjects[0])
        #  print(subject.head)
       

[]
[Baghdad]
[Saturday]
Earlier Saturday Baghdad


In [0]:
displacy.render(nlp(str(text)), style='dep', jupyter = True, options = {'distance': 120})
#.head amod' or w.dep_=='nmod'
#gpe=[w for w in subject.head.lefts if w.dep_=='prep']
#npadvmod saturday

In [0]:
TEXT = [
    'Net income was $9.4 million compared to the prior year of $2.7 million.',
    'Revenue exceeded twelve billion dollars, with a loss of $1b.',
]
displacy.render(nlp(str(TEXT)), style='dep', jupyter = True, options = {'distance': 120})

displacy.render(nlp(str(TEXT)), style='ent', jupyter = True)


In [0]:
def extract_currency_relations(doc):
    # merge entities and noun chunks into one token
    spans = list(doc.ents) + list(doc.noun_chunks)
    for span in spans:
        span.merge()

    relations = []
    for money in filter(lambda w: w.ent_type_ == 'MONEY', doc): #taking entity money only
        if money.dep_ in ('attr', 'dobj'): #money attr=link between verb and it's complement , dobj= (gave raise)
            subject = [w for w in money.head.lefts if w.dep_ == 'nsubj'] 
            if subject:
                subject = subject[0]
                relations.append((subject, money))
        elif money.dep_ == 'pobj' and money.head.dep_ == 'prep':
            relations.append((money.head.head, money))
    return relations
      # Expected output:
    # Net income      MONEY   $9.4 million
    # the prior year  MONEY   $2.7 million
    # Revenue         MONEY   twelve billion dollars
    # a loss          MONEY   1b

In [0]:
nlp = spacy.load('en')
docs = [nlp(t) for t in TEXTS]
for i, doc in enumerate(docs):
    for j, sent in enumerate(doc.sents):
        subjects = [w for w in sent if w.dep_ == 'nsubj']
        for subject in subjects:
            numbers = [w for w in subject.lefts if w.dep_ == 'amod']
            if len(numbers) == 1:
                print('document.sentence: {}.{}, subject: {}, action: {}, numbers: {}'.format(i, j, subject.text, subject.head.text, numbers[0].text))

In [0]:
nlp = spacy.load('en')
docs = [nlp(t) for t in TEXTS]
for i, doc in enumerate(docs):
    for j, sent in enumerate(doc.sents):
      print(i,doc)
      print(j,sent)

0 On Thursday there was a massive U.S. aerial bombardment in which more than 300 Tomahawk cruise missiles rained down on Baghdad.
0 On Thursday there was a massive U.S. aerial bombardment in which more than 300 Tomahawk cruise missiles rained down on Baghdad.
1 Earlier Saturday ,Baghdad was again targeted ..
0 Earlier Saturday ,Baghdad was again targeted ..


In [0]:
ex=nlp("US aerial Bombardment")
ex2=nlp("Bhaghdad was again Targeted")
for word in ex:
	print((word.text,word.tag_,word.pos_,word.dep_))
for word in ex.ents:
	print(word.text,word.label_)
for word in ex2:
	print((word.text,word.tag_,word.pos_,word.dep_))
for word in ex2.ents:
	print(word.text,word.label_)
spacy.explain('GPE')

('US', 'NNP', 'PROPN', 'nmod')
('aerial', 'JJ', 'ADJ', 'amod')
('Bombardment', 'NN', 'NOUN', 'ROOT')
US GPE
Bombardment GPE
('Bhaghdad', 'NNP', 'PROPN', 'nsubj')
('was', 'VBD', 'VERB', 'ROOT')
('again', 'RB', 'ADV', 'advmod')
('Targeted', 'VBN', 'VERB', 'acomp')
Bhaghdad GPE
Targeted GPE


'Countries, cities, states'

In [0]:
ex1='On Thursday there was a massive U.S. aerial bombardment in which more than 300 Tomahawk cruise missiles rained down on Baghdad.'
ex3='Earlier Saturday ,Baghdad was again targeted ..'
exx=nlp(ex1)
exx2=nlp(ex3)
for word in exx:
	print((word.text,word.tag_,word.pos_,word.dep_))
for word in exx.ents:
	print(word.text,word.label_)
for word in exx2:
	print((word.text,word.tag_,word.pos_,word.dep_))
for word in exx2.ents:
	print(word.text,word.label_)
spacy.explain('GPE')

('On', 'IN', 'ADP', 'prep')
('Thursday', 'NNP', 'PROPN', 'pobj')
('there', 'EX', 'ADV', 'expl')
('was', 'VBD', 'VERB', 'ROOT')
('a', 'DT', 'DET', 'det')
('massive', 'JJ', 'ADJ', 'amod')
('U.S.', 'NNP', 'PROPN', 'nmod')
('aerial', 'JJ', 'ADJ', 'amod')
('bombardment', 'NN', 'NOUN', 'attr')
('in', 'IN', 'ADP', 'prep')
('which', 'WDT', 'ADJ', 'pobj')
('more', 'JJR', 'ADJ', 'amod')
('than', 'IN', 'ADP', 'quantmod')
('300', 'CD', 'NUM', 'nummod')
('Tomahawk', 'NNP', 'PROPN', 'compound')
('cruise', 'NN', 'NOUN', 'compound')
('missiles', 'NNS', 'NOUN', 'nsubj')
('rained', 'VBD', 'VERB', 'relcl')
('down', 'RP', 'PART', 'prt')
('on', 'IN', 'ADP', 'prep')
('Baghdad', 'NNP', 'PROPN', 'pobj')
('.', '.', 'PUNCT', 'punct')
Thursday DATE
U.S. GPE
more than 300 CARDINAL
Tomahawk PRODUCT
Baghdad GPE
('Earlier', 'JJR', 'ADJ', 'amod')
('Saturday', 'NNP', 'PROPN', 'npadvmod')
(',', ',', 'PUNCT', 'punct')
('Baghdad', 'NNP', 'PROPN', 'nsubjpass')
('was', 'VBD', 'VERB', 'auxpass')
('again', 'RB', 'ADV', 'advm

'Countries, cities, states'

In [0]:
def extract_relations(text):
    docs = [nlp(t) for t in text]
    r=''
    for i, doc in enumerate(docs):
        for j, sent in enumerate(doc.sents):
            subjects = [w for w in sent if w.dep_ == 'attr'] #BOMBARDMENT
            #print(subjects)
            for subject in subjects:
              gpe=[w for w in subject.lefts if w.pos_=='PROPN'] #US
              #print(gpe)
              adj=[w for w in subject.lefts if w.dep_=='amod' and w.pos_=='ADJ'] #AERIAL
              he=[w for w in subject.head.lefts if w.dep_=='prep'] #ON
              #print(he)
              if he:
                  he=he[0]
                  da=[w for w in he.rights] #thursday
                  if da:
                      da=da[0]
                #print(da)
              if gpe:
                    gpe=gpe[0]
            #ge=[w for w in gpe.rights if w.pos_=='ADJ']
              if adj:
                    adj=adj[1]
    r=[he,da,gpe,a,subject]
    return(r)
rr=extract_relations(TEXTS)

In [0]:
#print(rr)
for i in rr:
  print(i,'',end='')

On Thursday U.S. aerial bombardment 

In [0]:
displacy.render(nlp(str(rr)), style='ent', jupyter = True)

In [0]:
def hashnum(day):
    # [Mon..Sun] -> [0..6]
    hashweekdays = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6}
    if k in hashweekdays.keys():
        num=(hashweekdays[k])
    return num
def updatedate(rr):
    doc=nlp(str(rr))
    dict=[[x.label_,x.text] for x in doc.ents]
    day=''
    for i in dict:
        if (i[0]=='DATE'):
            day=i[1]
    date_num=hashnum(day)
    import datefinder #
    import datetime #
    date="25 Nov 2018 (Sunday)"
    d1=list(datefinder.find_dates(date,source=False,strict=False))	
    d1=d1[0]
    #print(d1)
    #or
    #import dateparser
    #d2=dateparser.parse(date)
    #print(d2)
    d1_num=d1.day
    rd=d1_num-date_num
    new_date=d1.replace(day=rd)
    date=new_date.strftime('%d %b %y %a')
    return(date)
ud=updatedate(rr)
print(ud)

22 Nov 18 Thu


In [0]:
doc=nlp(str(rr))
d2=[[x.label_,x.text] for x in doc.ents]
#print(d2)

In [0]:
k=''
for i in d2:
  if i[0]=='DATE':
    k=i[1]
#print(k)

Thursday


In [0]:
# [Mon..Sun] -> [0..6]
hashweekdays = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6}

In [0]:
if k in hashweekdays.keys():
  num=(hashweekdays[k])
#print(num)

3


In [0]:
current_date=25
de=current_date-num
print(de)

22


In [0]:
Dt="%i Nov 2018"%de
print(Dt)


22 Nov 2018


In [0]:
#get published date with help of datefinder

import datefinder
date="25 Nov 2018 (Sunday)"
m=list(datefinder.find_dates(date,source=False,strict=False))
#for m in m:
print(m[0])
mm=m[0]
#or

import dateparser
d2=dateparser.parse(date)
print(d2)

2018-11-25 00:00:00
2018-11-25 00:00:00


In [0]:
D=d2.day
print(D)

25


In [0]:
d3=d2.replace(day=de)
d3=d3.strftime('%d %b %y %a')
print(d3)

22 Nov 18 Thu


In [0]:
import datetime
dt=mm.strftime('%d %b %y %a')
print(dt)

25 Nov 18 Sun


In [0]:
t=mm.timetuple()
print(t)

time.struct_time(tm_year=2018, tm_mon=11, tm_mday=25, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=6, tm_yday=329, tm_isdst=-1)


In [0]:
l=[]
for i in t:
  l.append(i)
print(l)
l[2]=de
print(l)
#tt=tuple(l)
#print(tt)
#dt=datetime.date(tt)
#print(dt)

[2018, 11, 25, 0, 0, 0, 6, 329, -1]
[2018, 11, 22, 0, 0, 0, 6, 329, -1]


TypeError: ignored

In [0]:
def hashnum(day):
	# [Mon..Sun] -> [0..6]
	hashweekdays = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6}
	if k in hashweekdays.keys():
  		num=(hashweekdays[k])
	return num
def updatedate(rr):
	doc=nlp(str(rr))
	dict=[[x.label_,x.text] for x in doc.ents]
	day=''
	for i in dict:
  		if i[0]=='DATE':
    		day=i[1]
	date_num=hashnum(day)
	import datefinder
	date="25 Nov 2018 (Sunday)"
	m=list(datefinder.find_dates(date,source=False,strict=False))
	#for m in m:
		#print(m[0])
		mm=m[0]
	#or
	import dateparser
	d2=dateparser.parse(date)
	print(d2)
